In [250]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from BTplatform import DataLoader
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import math
from IPython.display import FileLink
import seaborn as sns
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector

In [280]:
class EDA:
    def __init__(self, DataLoader):
        self.loader = DataLoader

    def getACF(self, save = False, graph = True, lag = 20, ncols = 3, conf = True):
        '''
        returns the ACFs of all stocks at this timestep for the last 20 lags and graphs all of them.
        '''
        # error handling
        if lag < 1 or isinstance(lag, int) == False:
            raise ValueError('lag must be integer greater than 1')

        # get data and making acfs array
        data = self.loader.returnsToNow() # all returns of required stocks for last lags + today
        acfs = []

        if graph:
            ncols = max(1, int(ncols))
            nrows = math.ceil(self.loader.nins / ncols)
            fig, axes = subplots(nrows, ncols, figsize = (4 * ncols, 3 * nrows), sharex = True, sharey = True) # same x/y axis
            axes = np.atleast_1d(axes).ravel() # atleast_1d --> if its 1d it makes it 2d by chucking brackets around it
            # ravel so can go axes[i] and put them one by one

        for i in range(self.loader.nins):
            stock = data[:, i] # stock data for one stock

            # getting acfs and confidence intervals
            acf_vals = acf(stock, nlags = lag, fft=False, missing='conservative') # dont take shortcut and take out NAs
            acfs.append(acf_vals)

            if graph:
                ax = axes[i]
                ax.axhline(0, c = 'k', lw = 0.5)
                ax.bar(np.arange(lag + 1), acf_vals, width = 0.3, align = 'center') # align puts it at the lag (center)
                ax.plot(acf_vals, lw = 0.5, c = 'r')
                if conf:
                    bound = 1.96 / np.sqrt(len(stock))
                    ax.fill_between(np.arange(lag + 1), -bound, bound, alpha = 0.1, color = 'm')
                ax.set_title(f'Stock {self.loader.stocks[i]}')
                ax.set_xlabel('Lags')
                ax.set_ylabel('ACF')
                ax.set_xlim(0, lag)
        if graph:
            for j in range(self.loader.nins, len(axes)):
                fig.delaxes(axes[j]) # removes the axes that weren't used
            fig.tight_layout() # adjust so titles dont collide and squeeze shit in
            fig.subplots_adjust(top = 1) # leave room for title

            # if saving file for algo presentation
            if save:
                fig.savefig("ACF_plots.png")
                print(FileLink("ACF_plots.png"))
                
        return np.vstack(acfs)

    def getPACF(self, save = False, graph = True, lag = 20, ncols = 3, conf = True):
        '''
        returns the pACFs of all stocks at this timestep for the last 20 lags and graphs all of them.
        '''
        # error handling
        if lag < 1 or isinstance(lag, int) == False:
            raise ValueError('lag must be integer greater than 1')

        # get data and making acfs array
        data = self.loader.returnsToNow() # all returns of required stocks for last lags + today
        pacfs = []

        if graph:
            ncols = max(1, int(ncols))
            nrows = math.ceil(self.loader.nins / ncols)
            fig, axes = subplots(nrows, ncols, figsize = (4 * ncols, 3 * nrows), sharex = True, sharey = True) # same x/y axis
            axes = np.atleast_1d(axes).ravel() # atleast_1d --> if its 1d it makes it 2d by chucking brackets around it
            # ravel so can go axes[i] and put them one by one

        for i in range(self.loader.nins):
            stock = data[:, i] # stock data for one stock

            # getting acfs and confidence intervals
            pacf_vals = pacf(stock, nlags = lag) # dont take shortcut and take out NAs
            pacfs.append(pacf_vals)

            if graph:
                ax = axes[i]
                ax.axhline(0, c = 'k', lw = 0.5)
                ax.bar(np.arange(lag + 1), pacf_vals, width = 0.3, align = 'center') # align puts it at the lag (center)
                ax.plot(pacf_vals, lw = 0.5, c = 'r')
                if conf:
                    bound = 1.96 / np.sqrt(len(stock))
                    ax.fill_between(np.arange(lag + 1), -bound, bound, alpha = 0.1, color = 'm')
                ax.set_title(f'Stock {self.loader.stocks[i]}')
                ax.set_xlabel('Lags')
                ax.set_ylabel('PACF')
                ax.set_xlim(0, lag)
        if graph:
            for j in range(self.loader.nins, len(axes)):
                fig.delaxes(axes[j]) # removes the axes that weren't used
            fig.tight_layout() # adjust so titles dont collide and squeeze shit in
            fig.subplots_adjust(top = 1) # leave room for title

            # if saving file for algo presentation
            if save:
                fig.savefig("PACF_plots.png")
                print(FileLink("PACF_plots.png"))
                
        return np.vstack(pacfs)

    def getADF(self, log = True):
        '''
        uses adf to check for weak stationality
        '''
        returnSeries = self.loader.returnsToNow(log).T # getting returns til now (set t to end)
        
        print('ADF statistics!!')
        for i, price in enumerate(returnSeries):
            results = adfuller(price) # gettting ts, read docs
            
            p = results[1]
            if p < 0.05:    
                print(f'''# Stock {self.loader.stocks[i]} ####################
                ADF statistic: {results[0]}
                Mackinnon p value: {results[1]}
                Observations used: {results[3]}
                Crit. values: {results[4]}''') # mackinnon p value make sure < 0,05

    def plotReturns(self, log = True, ncols = 1):
        '''
        plots the returns of all stocks in dataloader -> useful to check if further differencing is required, visual conf of adf.
        '''
        returnSeries = self.loader.returnsToNow(log).T # getting returns til now (set t to end)
        ncols = max(1, ncols)
        nrows = math.ceil(self.loader.nins/ncols)
        fig, axes = subplots(nrows, ncols, figsize = (16 * ncols, 3 * nrows), sharex = True, sharey = True)
        axes = np.atleast_1d(axes).ravel()
        
        for i, price in enumerate(returnSeries):
            ax = axes[i]
            ax.plot(price)
            ax.set_title(f'Stock {self.loader.stocks[i]}')
            ax.axhline(0, c = 'r', ls = '--')

        for j in range(self.loader.nins, len(axes)):
            fig.delaxes(axes[j]) # removes the axes that weren't used
            fig.tight_layout() # adjust so titles dont collide and squeeze shit in
            fig.subplots_adjust(top = 1) # leave room for title

    def fitARIMA(self, params, seasonal_params = None, train_len = 0.8, ncols = 1, log = True):
        '''
        params is (p, d, q)
        Fits arma model and plots the predicted vs test
        '''
        returnSeries = self.loader.returnsToNow(log).T # getting returns til now (set t to the end)

        for i, price in enumerate(returnSeries):
            n_train = int(train_len) if train_len > 1 else int(self.loader.t * train_len)
            train = price[:n_train]
            test = price[n_train:]

            #speed things up if statement so dont have to go thorugh sarima bullshit
            if seasonal_params == None:
                model = SARIMAX(train, order = params).fit(disp=False)
            else:
                model = SARIMAX(train, order = params, seasonal_order = seasonal_params).fit(disp=False)
            print(f'''
            # Stock {self.loader.stocks[i]}  ####################
            {model.summary()}
            ''')
            pred = model.predict(start=n_train, end=self.loader.t, dynamic=False) # dynamic = false if we know past values

    def backtestARIMA(self, params, seasonal_params = [0, 0, 0, 1], train_len = 0.8, warning = False, save = True, ncols = 1, log = True):
        '''
        same settings as fitARIMA
        arima plotter
        backtests and gets root mean square error
        '''
        returnSeries = self.loader.returnsToNow(log).T # getting returns

        # graph matrix settings
        ncols = max(1, ncols)
        nrows = math.ceil(self.loader.nins/ncols)
        fig, axes = subplots(nrows, ncols, figsize = (16 * ncols, 3 * nrows), sharex = True)
        axes = np.atleast_1d(axes).ravel()
        
        for i, price in enumerate(returnSeries):
            ax = axes[i]

            # initial training window
            n_train = int(train_len) if train_len > 1 else int(self.loader.t * train_len)

            preds = []
            errors = []
            n_errors = []
            for t in range(n_train, self.loader.t):
                y_train = price[:t]

                # getting rid of stupid warnings 
                if warning == False:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore", category=ConvergenceWarning)
                        warnings.simplefilter("ignore", category=RuntimeWarning)

                        #speed things up if statement so dont have to go thorugh sarima bullshit
                        if seasonal_params == None:
                            model = SARIMAX(train, order = params).fit(disp=False)
                        else:
                            model = SARIMAX(train, order = params, seasonal_order = seasonal_params).fit(disp=False)
                else:
                    #speed things up if statement so dont have to go thorugh sarima bullshit
                    if seasonal_params == None:
                        model = SARIMAX(train, order = params).fit(disp=False)
                    else:
                        model = SARIMAX(train, order = params, seasonal_order = seasonal_params).fit(disp=False)
                pred = model.predict(start = len(y_train), end = len(y_train), dynamic = False) # dynamic = false if we know past values
                preds.append(pred)
                true_val = price[t]
                errors.append(true_val - pred)
                n_errors.append(true_val - np.mean(y_train))

            ax.plot(price[:n_train + 1])
            ax.plot(range(n_train, self.loader.t), price[n_train:], color = 'silver')
            ax.plot(range(n_train, self.loader.t), preds, color = 'orange')
            ax.set_title(f'Stock {self.loader.stocks[i]}')
            
            # change error arrays to numpy so can do vector shi
            errors = np.array(errors)
            n_errors = np.array(n_errors)

            # error metric calcs
            mae = np.mean(np.abs(errors)) # mean absolute v error kinda useless but chatgpt said include it
            rmse = np.sqrt(np.mean(errors**2)) # rmse NOT normalized
            mrmse = rmse/np.std(price[n_train:]) # normalise on std of testing window
            naive_rmse = 1 - rmse/np.sqrt(np.mean(n_errors ** 2)) # percentage improvement compared to just guessing mean

            print(f'# Stock {self.loader.stocks[i]} ####################')
            print(f"MAE:  {mae:.6f}")
            print(f"RMSE: {rmse:.6f}")
            print(f"norm. RMSE: {mrmse:.6f}")
            print(f'vs. 0 baseline: {naive_rmse:.6f}')

        if save:
            fig.savefig('ARIMA_Backtests.png')

    def topCorrelated(self, stocknum, dropFirst = False, log = True):
        '''
        returns top correlated stocks to stock i 
        '''
        # changing stocknum into an index that we can use
        try:
            i = self.loader.stocks.index(stocknum)
        except:
            raise ValueError('enter a stock thats IN the database')

        returns = self.loader.returnsToNow(log).T # getting returns
        corrArray = np.corrcoef(returns)[i] # getting correlation matrix for it
        corrReturn = []
        for n in range(len(corrArray)):
            corrReturn.append([corrArray[n], self.loader.stocks[n]])

        corrReturn.sort(key=lambda x: abs(x[0]), reverse = True) # Sorting by highest to lowest now
        if dropFirst:
            corrReturn = corrReturn[1:] # dropping first one (IF HIGHLY CORRELATED STOCK (1) THEN DONT TAKE THIS)
        print(f'## Top Correlating Assets for Stock {stocknum} ####################')
        for corr in corrReturn:
            print(f'Stock {corr[1]}: {corr[0]}')
        
    def plotPrices(self):
        '''
        just plots the prices
        '''
        fig, ax = subplots(figsize = (8, 8))
        for i in range(self.loader.nins):
            ax.plot(self.loader.data.T[i], label = f'Stock {self.loader.stocks[i]}')
        ax.legend()
        ax.set_title('All Asset Prices')
        ax.set_xlabel('Time')
        ax.set_ylabel('Price')

    def fitLinReg(self, stocknum, lag = 1, cv = 10, metric = 'r2'):
        '''
        uses mlxtend to find best subset via greedy subset selection (type of forward/backward)
        outputs regression for stocknum --> SET LOADER t TO END! (uses cv)
        change r2 if classification --> to accuracy
        '''
        regData = pd.DataFrame(self.loader.getLagFeatures(stocknum, lag = lag)).dropna()

        # getting x and y
        y, x = regData.iloc[:, 0], regData.iloc[:, 1:]

        # selecting best features
        sfs = SequentialFeatureSelector(linear_model.LinearRegression(), k_features = (1, self.loader.nins), forward = True, scoring = metric, cv = cv)
        features = [int(name) for name in sfs.fit(x, y).k_feature_names_] # turning all to ints

        # fitting regression
        x_selection = regData.iloc[:, features]
        x_train, x_test, y_train, y_test = train_test_split(x_selection, y, test_size = math.ceil(self.loader.t / cv))

        x_train = sm.add_constant(x_train, has_constant = 'add') # adding intercept shi manually
        x_test = sm.add_constant(x_test, has_constant = 'add')

        model = sm.OLS(y_train, x_train).fit()
        print(model.summary())
        
        y_pred = model.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        print(f'R2: {r2}')
        
        

In [282]:
prices = np.loadtxt('../Data/StockData.txt')
prices

array([[13.46, 71.65, 48.46, ..., 36.22, 49.  , 56.09],
       [13.48, 72.1 , 48.52, ..., 36.27, 48.84, 56.08],
       [13.47, 72.35, 48.48, ..., 36.39, 48.56, 55.9 ],
       ...,
       [ 8.33, 69.68, 43.38, ..., 30.07, 59.42, 56.23],
       [ 8.25, 70.15, 43.35, ..., 30.07, 59.56, 56.21],
       [ 8.26, 70.93, 43.34, ..., 29.91, 59.12, 56.32]])

In [286]:
data = DataLoader(prices)
data.goToEnd()

eda = EDA(data)
for i in range(50):
    eda.fitLinReg(i)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.384
Date:                Sat, 06 Sep 2025   Prob (F-statistic):            0.00260
Time:                        11:02:43   Log-Likelihood:                 4105.0
No. Observations:                1123   AIC:                            -8196.
Df Residuals:                    1116   BIC:                            -8161.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.000     -2.239      0.0

In [ ]:
x = regData.iloc[:, 1:]
y = regData.iloc[:, 0]

sfs = SequentialFeatureSelector(linear_model.LinearRegression(), k_features = (1, 50), forward = True, scoring = 'r2', cv = 10)

selected_features = sfs.fit(x, y)

In [131]:
pd.DataFrame(y)

,0
0,-0.000742
1,0.004444
2,0.008097
3,-0.001467
4,0.003664
...,...
1243,-0.018802
1244,-0.004756
1245,-0.007177
1246,-0.009650


In [209]:
[0] + list(selected_features.k_feature_names_)

[0, 5, 7, 9, 14, 18, 28]

In [163]:
pd.DataFrame(regData)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,-0.000742,0.001485,0.006261,0.001237,-0.000396,-0.000768,-0.003854,-0.001582,0.002721,0.003448,...,-0.003683,0.003759,-0.001384,0.014653,-0.001909,-0.000760,-0.002639,0.001380,-0.003271,-0.000178
1,0.004444,-0.000742,0.003461,-0.000825,0.002373,-0.005007,-0.012432,0.001582,0.002922,-0.001866,...,-0.001231,0.006589,0.004147,-0.012958,0.006800,-0.001713,0.005675,0.003303,-0.005750,-0.003215
2,0.008097,0.004444,0.002209,-0.001238,0.002565,-0.002706,-0.010216,-0.001054,0.015716,-0.000862,...,0.003381,-0.000355,-0.006920,-0.005265,0.001354,-0.001716,-0.001213,0.000549,0.009020,0.004284
3,-0.001467,0.008097,-0.007197,-0.000413,-0.001972,0.005983,-0.003165,-0.003699,0.002868,0.002011,...,0.001533,0.000000,-0.002782,0.004756,0.002433,0.000763,0.007459,0.000275,-0.017707,-0.004284
4,0.003664,-0.001467,0.004988,0.000207,-0.002570,-0.000770,-0.007955,0.002115,-0.007805,0.001864,...,-0.003376,-0.007843,-0.007690,-0.005949,0.004849,-0.000763,0.009395,-0.001374,0.001868,-0.011515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,-0.018802,0.008182,0.003598,-0.001150,0.001503,-0.001120,0.003992,-0.003997,0.010641,-0.002726,...,-0.006562,0.000271,-0.001369,-0.001741,0.009211,-0.003673,-0.001565,0.001004,-0.006936,-0.005513
1244,-0.004756,-0.018802,0.000574,0.000460,0.001501,-0.000747,-0.001994,-0.002674,0.000504,-0.002125,...,-0.003103,0.005273,-0.002743,-0.021137,-0.004266,0.000000,0.004558,0.002338,0.015663,0.004448
1245,-0.007177,-0.004756,0.007011,-0.000921,0.000500,0.000560,0.000000,-0.002010,-0.005219,-0.001978,...,0.005811,0.001348,-0.008276,0.023748,-0.007923,-0.001842,0.000390,0.001000,-0.005530,-0.000888
1246,-0.009650,-0.007177,-0.006580,-0.000922,0.000250,0.005773,0.003984,-0.000671,-0.006096,-0.001829,...,-0.001546,0.002153,0.001384,0.009085,0.010551,-0.001845,0.008664,0.001997,-0.001513,-0.000889
